<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/ner-fine-tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
import pandas as pd
import numpy as np

In [1]:
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/train.txt'
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/test.txt'
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/dev.txt'
!mkdir data && mv train.txt data && mv test.txt data && mv dev.txt data

train.txt           100%[===================>]   2.20M  --.-KB/s    in 0.07s   
test.txt            100%[===================>] 108.10K  --.-KB/s    in 0.01s   
dev.txt             100%[===================>] 160.83K  --.-KB/s    in 0.01s   


# Benchmark2

In [3]:
!cat data/train.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > train.txt.tmp
!cat data/test.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > test.txt.tmp
!cat data/dev.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > dev.txt.tmp

In [4]:
!git clone https://github.com/huggingface/transformers
!pip install transformers/
!pip install sentencepiece

Cloning into 'transformers'...
remote: Enumerating objects: 60842, done.
remote: Total 60842 (delta 0), reused 0 (delta 0), pack-reused 60842
Receiving objects: 100% (60842/60842), 45.57 MiB | 23.43 MiB/s, done.
Resolving deltas: 100% (42970/42970), done.
Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 7.8MB/s 
     |████████████████████████████████| 2.9MB 23.3MB/s 
  Created wheel for transformers: filename=transformers-4.3.0.dev0-cp36-none-any.whl size=1783316 sha256=8e3b3fc4b65dba3d2b05d712bd54a5a0b5881a29ccfbea4de7af844ff5dcf1ba
  Stored in directory: /tmp/pip-ephem-wheel-cache-s3iy9msc/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=96df2797bc16d4b57f5f6d7419cc95fcec8297ea1fc7bbc621a5ee2c1

In [5]:
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

--2021-01-28 05:39:07--  https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 991 [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]     991  --.-KB/s    in 0s      

2021-01-28 05:39:07 (65.5 MB/s) - ‘preprocess.py’ saved [991/991]



In [6]:
!python3 preprocess.py dev.txt.tmp HooshvareLab/bert-base-parsbert-uncased 128 > dev.txt
!python3 preprocess.py train.txt.tmp HooshvareLab/bert-base-parsbert-uncased 128 > train.txt
!python3 preprocess.py test.txt.tmp HooshvareLab/bert-base-parsbert-uncased 128 > test.txt

Downloading: 100% 434/434 [00:00<00:00, 362kB/s]
Downloading: 100% 1.22M/1.22M [00:00<00:00, 3.15MB/s]


In [7]:
!pip install -r transformers/examples/token-classification/requirements.txt

     |████████████████████████████████| 51kB 3.9MB/s 
     |████████████████████████████████| 163kB 11.2MB/s 
     |████████████████████████████████| 245kB 13.8MB/s 
     |████████████████████████████████| 20.7MB 1.3MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=e513bbfc1a4f0b46f9c4614e20ab6b315c1eb8994aad715b8a9af30b28072fec
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [8]:
!pip install conllu

In [9]:
!mkdir processed_data
!mv train.txt processed_data/ && mv test.txt processed_data/ && mv dev.txt processed_data/

In [10]:
!cat processed_data/train.txt processed_data/test.txt processed_data/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

In [11]:
!python3 transformers/examples/legacy/token-classification/run_ner.py --data_dir processed_data/ \
--labels labels.txt \
--model_name_or_path HooshvareLab/bert-base-parsbert-uncased \
--output_dir eval/ \
--max_seq_length  128 \
--save_steps 750 \
--seed 1 \
--do_train \
--do_eval \
--do_predict

2021-01-28 05:40:19.565378: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
01/28/2021 05:40:21 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
01/28/2021 05:40:21 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=eval/, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=EvaluationStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Jan28_05-40-21_c389095872f4, logging_first_step=False, logging_steps=500, save_steps=750, save_total_limit=None, no_cu

In [13]:
!zip -r eval.zip eval

  adding: eval/ (stored 0%)
  adding: eval/checkpoint-2250/ (stored 0%)
  adding: eval/checkpoint-2250/scheduler.pt (deflated 49%)
  adding: eval/checkpoint-2250/config.json (deflated 54%)
  adding: eval/checkpoint-2250/pytorch_model.bin (deflated 8%)
  adding: eval/checkpoint-2250/optimizer.pt (deflated 41%)
  adding: eval/checkpoint-2250/trainer_state.json (deflated 60%)
  adding: eval/checkpoint-2250/training_args.bin (deflated 45%)
  adding: eval/test_predictions.txt (deflated 71%)
  adding: eval/test_results.txt (deflated 34%)
  adding: eval/config.json (deflated 54%)
  adding: eval/checkpoint-750/ (stored 0%)
  adding: eval/checkpoint-750/scheduler.pt (deflated 49%)
  adding: eval/checkpoint-750/config.json (deflated 54%)
  adding: eval/checkpoint-750/pytorch_model.bin (deflated 8%)
  adding: eval/checkpoint-750/optimizer.pt (deflated 41%)
  adding: eval/checkpoint-750/trainer_state.json (deflated 45%)
  adding: eval/checkpoint-750/training_args.bin (deflated 45%)
  adding: eval/

In [14]:
!zip -r runs.zip runs

  adding: runs/ (stored 0%)
  adding: runs/Jan28_05-40-21_c389095872f4/ (stored 0%)
  adding: runs/Jan28_05-40-21_c389095872f4/events.out.tfevents.1611840450.c389095872f4.263.2 (deflated 29%)
  adding: runs/Jan28_05-40-21_c389095872f4/events.out.tfevents.1611812466.c389095872f4.263.0 (deflated 54%)
  adding: runs/Jan28_05-40-21_c389095872f4/1611812466.8290157/ (stored 0%)
  adding: runs/Jan28_05-40-21_c389095872f4/1611812466.8290157/events.out.tfevents.1611812466.c389095872f4.263.1 (deflated 59%)


In [15]:
!ls -lh

total 4.4G
drwxr-xr-x  2 root root 4.0K Jan 28 05:38 data
-rw-r--r--  1 root root 161K Jan 28 05:38 dev.txt.tmp
drwxr-xr-x  5 root root 4.0K Jan 28 13:29 eval
-rw-r--r--  1 root root 4.4G Jan 28 13:49 eval.zip
-rw-r--r--  1 root root   74 Jan 28 05:39 labels.txt
-rw-r--r--  1 root root  991 Jan 28 05:39 preprocess.py
drwxr-xr-x  2 root root 4.0K Jan 28 13:27 processed_data
drwxr-xr-x  3 root root 4.0K Jan 28 05:41 runs
-rw-r--r--  1 root root 4.8K Jan 28 13:49 runs.zip
drwxr-xr-x  1 root root 4.0K Jan 20 17:27 sample_data
-rw-r--r--  1 root root 109K Jan 28 05:38 test.txt.tmp
-rw-r--r--  1 root root 2.3M Jan 28 05:38 train.txt.tmp
drwxr-xr-x 16 root root 4.0K Jan 28 05:38 transformers
